In [1]:
import numpy as np
import random
import os
import openai
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
#from .. import *

import json
from  StorySquadAI.contestant import StorySquadAI
HoaxAI = StorySquadAI(data_dir="..//..//data//")
bot_default = HoaxAI.create_bot_with_personality(personality="originaltestbot")
bot_default.engine_to_use='ada'

bot_bubblebot_ada = HoaxAI.create_bot_with_personality(personality="bubblebot")
bot_bubblebot_ada.engine_to_use='ada'

bot_buzzkillbot_ada = HoaxAI.create_bot_with_personality(personality="bubblebot")
bot_buzzkillbot_ada.engine_to_use= 'ada'

bot_bubblebot_curie = HoaxAI.create_bot_with_personality(personality="bubblebot")
bot_bubblebot_curie.engine_to_use='curie'

bot_bubblebot_v2_curie = HoaxAI.create_bot_with_personality(personality="bubblebot_v2")
bot_bubblebot_v2_curie.engine_to_use='curie'


bot_buzzkillbot_curie = HoaxAI.create_bot_with_personality(personality="bubblebot")
bot_buzzkillbot_curie.engine_to_use='curie'


orig_context_movie = bot_default.personality.responses["movie"].context_doc
orig_context_person = bot_default.personality.responses["person"].context_doc
orig_context_thing = bot_default.personality.responses["thing"].context_doc

#context_dir = '../../data'
#orig_context_movie = open(os.path.join(context_dir, "movie.context.txt")).read()
#orig_context_person = open(os.path.join(context_dir, "person.context.txt")).read()
#orig_context_thing = open(os.path.join(context_dir, "thing.context.txt")).read()

test_set = [("yes","no"),("good","bad"),("up","down"),("north","south"),("hard","soft"),("liquid","solid"),("summer","winter"),("eutopia","dystopia")]
test_set_list = ["yes","no","good","bad","up","down","north","south","hard","soft","liquid","solid","summer","winter","eutopia","dystopia"]

default_query_list=[
    ('apple','The Divinci Code','John Dillon'),
    ('dompteuse','blackfish','Catrin Saeed'),
    ('saggar','In the Name of the Father','Catrin Saeed'),
    ('potometer','Billy Elliot','Mohammad Rosmunda'),
    ('amoretto','Jeff, Who Lives at Home','Amis Shiva'),
    ('pergola','Calamity Jane','Ronan Kara'),
    ('dasypoedes','Monster-In-Law','Huangdi Mohammad'),
    ('selenography','The Departed ','Tamela Ace'),
    ('scrimure','The Pacifier','Yvonne Tatton'),
    ('paronomasia','Aquamarine','Dareios Katarina'),
    ('doxastic','Fury','Marijana Pau'),
    ('brank','What Dreams May Come','Cadeyrn Manish'),
]

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def get_embedding(bot:StorySquadAI.StorySquadBot,s:str="test string"):
    embedding = openai.Engine(id=f"{bot.engine_to_use}-similarity").embeddings(
    input=s
    )['data'][0]['embedding']
    return embedding

def get_embeddings_for_list(bot:StorySquadAI.StorySquadBot,l:list):
    return [get_embedding(bot,c) for c in l]

def get_set_of_colors(min_sat:float,min_dist:float,number:int,set_size:int=1):
    o=np.empty((1,3))
    while len(o) < (number*set_size)+2:
            s=0;v=1;set_min_dist=0
            try_count = 1000
            while (s < min_sat) | (v>.55) | (set_min_dist<min_dist):
                new_rgb=np.random.random((1,3))
                s = (abs(new_rgb[0,0]-new_rgb[0,1])+abs(new_rgb[0,0]-new_rgb[0,2])+abs(new_rgb[0,1]-new_rgb[0,2]))/3
                v = np.mean(new_rgb)
                set_min_dist = np.sum(abs(o - new_rgb),axis=1).min()
                try_count=try_count-1
                if try_count <=0:
                    break
                #print(s,v,set_min_dist)
            for _ in range (set_size):
                o=np.concatenate([o,new_rgb])
    return o[1:(number*set_size)+1].tolist()

class ExperimentalPCA:

    def __init__(self):
        from  sklearn.decomposition import PCA
        self.PCA = PCA(n_components=2)

    def get_scatter_pts_for_embeddings_list(self,embeddings,fit=False):
        if fit:
            data = self.PCA.fit_transform(embeddings)
        else:
            data = self.PCA.transform(embeddings)

        x = [x for x,y in data]
        y = [y for x,y in data]
        return x, y

def get_embeddings_for_list_of_context_docs(context_doc_list:list,marker_list:list=['o']):
    out_emb =[]
    out_markers=[]
    for i,context_doc in enumerate(context_doc_list):
        current_marker = marker_list[i]
        context_doc=context_doc.replace("U.S","US")
        for line in context_doc.splitlines():
            if not("Movie:" in line):
                if not("Who Is" in line):
                    if not("Q: " in line):
                        if not("C: " in line):
                            if len(line)> 2:
                                print(line)
                                out_emb.append(get_embedding(line))
                                out_markers.append(current_marker)

    return out_emb,out_markers

def generate_results(query_list:list,bot:StorySquadAI.StorySquadBot):
    results =[]
    results_embeddings=[]

    for query_tuple in query_list:
        thing_res = bot.thing(prompt= query_tuple[0])
        results.append(thing_res)
        results_embeddings.append(get_embedding(thing_res))

        person_res = bot.person(person= query_tuple[1])
        results.append(person_res)
        results_embeddings.append(get_embedding(person_res))

        movie_res = bot.movie(movie= query_tuple[2])
        results.append(movie_res)
        results_embeddings.append(get_embedding(movie_res))


    return results,results_embeddings

def get_plot_display_data_for_list(list_of_strings:str,bot:StorySquadAI.StorySquadBot,color:list,shape:str):
    """given a list of string generates a list of tuples containing the string, color, shape needed for plotting"""
    pass

In [ ]:
with open ("default_results.txt","r") as f:
    default_results = json.loads(f.read())
    default_results_thing= default_results[::3]
    default_results_person= default_results[1::3]
    default_results_movie= default_results[2::3]
with open ("bubble_bot_results.txt","r") as f:
    bubble_bot_result = json.loads(f.read())
    bubble_bot_result_thing= bubble_bot_result[::3]
    bubble_bot_result_person= bubble_bot_result[1::3]
    bubble_bot_result_movie= bubble_bot_result[2::3]

corpi_list=[test_set,
            orig_context_thing,orig_context_person,orig_context_movie,
            default_results_thing,default_results_person,default_results_movie,
            bubble_bot_result_thing,bubble_bot_result_person,bubble_bot_result_movie]

test_set_emb = get_embeddings_for_list(test_set_list)
embeddings_for_default_context_doc = get_embeddings_for_list_of_context_docs(
    [orig_context_movie,orig_context_person,orig_context_thing],
    marker_list=['t','o','s'])

all_emb = test_set_emb+\
          embeddings_for_default_context_doc+\
          default_result_embeddings+\
          bubble_bot_result_embeddings

test_set_colors = get_set_of_colors(min_dist=.55,min_sat=.5,number=8,set_size=2)
default_context_colors = [[0,0,0]]*len(embeddings_for_default_context_doc)
bubble_bot_res_scatter_colors = [[.75,.25,1.0,.15]]*len(bubble_bot_result_embeddings)
default_res_scatter_colors = [[.25,.5,1.0,.15]]*len(default_result_embeddings)

all_colors = test_set_colors+\
             default_context_colors+\
             default_res_scatter_colors+\
             bubble_bot_res_scatter_colors

In [ ]:
exp_PCA = ExperimentalPCA()

all_x,all_y =exp_PCA.get_scatter_pts_for_embeddings_list(all_emb,fit=True)

plt.figure(figsize=(15,15))
plt.scatter(all_x, all_y,c=all_colors,s=128,marker="s")

plt.title("test set embeddings")
plt.show()

In [ ]:
all_emb = test_set_emb+default_results+embeddings_for_default_context_doc
from  sklearn.decomposition import PCA
this_PCA = PCA(n_components=2)
fit = this_PCA.fit_transform(all_emb)


vis_dims = fit
x = [x for x,y in vis_dims]
y = [y for x,y in vis_dims]
plt.scatter(x,y)
plt.show()

In [ ]:
default_results,default_results_embeddings = generate_results(default_query_list)

In [ ]:
out = json.dumps(default_results)
with open ("default_results.txt","w") as f:
    f.write(out)

out = json.dumps(default_results_embeddings)
with open ("default_embeddings.txt","w") as f:
    f.write(out)

In [6]:
## load and shape data
files=["things_100.txt","names_100.txt","movies_100.txt"]
extended_query_list =[]
for file_name in files:
    with open(file_name) as f:
        extended_query_list.append(f.read().splitlines())
extended_query_list= zip(*extended_query_list)
extended_query_list = [ (a,b,c) for a,b,c in extended_query_list]
print(extended_query_list)
print(extended_query_list[1])

[('acaer', 'Rolando Morales', 'Car Keys'), ('strooms', 'Forest Cherry', 'Warehouse'), ('strustate', 'Mack Cline', 'Tear-Jerker'), ('fibbelous', 'Cornelius Anthony', 'Green Thumb'), ('hettickism', 'Omar Anderson', 'The Help'), ('nonsives', 'Rick Gates', 'Sugar Addict'), ('fewtert', 'Cecil Taylor', 'Telephone Tag'), ('adberastered', 'Terrance Myers', 'Fire Flies'), ('fruismyther', 'Xavier Dunlap', 'Random Wink'), ('deferts', 'Augustine Ruiz', 'Fancy Pants'), ('sineout', 'Lon Rich', 'Morning Sunrise'), ('mentics', 'Jacob Hart', 'The Glance'), ('appectedies', 'Bobbie Murphy', 'In The Shadows'), ('sedubing', 'Carlo Kramer', 'Magic Lamp'), ('canorting', 'Herbert Tucker', 'Peak Outside'), ('climpower', 'Salvador Flynn', 'Shopping Addict'), ('swooflia', 'Amado Hancock', 'Memory Lane'), ('berictions', 'Raymundo Giles', 'Backhanded Insult'), ('usness', 'Roberto Duran', 'Heroic Gesture'), ('relatern', 'Pat Hayes', 'The Voyage'), ('submisitted', 'Robt Ashley', 'Alarm Clock'), ('orousloth', 'Eduard

In [3]:
def get_responses_for_bot_for_query_list(bot,query_list):
    out_queries = []
    out_results =[]
    out_result_embeddings =[]
    for query_tuple in query_list:
        thing_res = bot.thing(prompt = query_tuple[0])
        out_result_embeddings.append(get_embedding(bot,thing_res))
        out_results.append(thing_res)
        out_queries.append(query_tuple[0])

        movie_res = bot.movie(movie = query_tuple[1])
        out_result_embeddings.append(get_embedding(bot,movie_res))
        out_results.append(movie_res)
        out_queries.append(query_tuple[1])

        person_res = bot.person(person = query_tuple[2])
        out_result_embeddings.append(get_embedding(bot,person_res))
        out_results.append(person_res)
        out_queries.append(query_tuple[2])

    return out_results,out_result_embeddings,out_queries

In [8]:
def create_save_bot_results_for_bot(bot:str,number:int=1,engine:str="ada"):
    _hoax_ai = StorySquadAI(data_dir="..//..//data//")
    _bot =_hoax_ai.create_bot_with_personality(bot)
    _bot.engine_to_use=engine
    _bot_results,_bot_embeddings,_ = \
        get_responses_for_bot_for_query_list(_bot,extended_query_list[0:number])


    for i,_list in enumerate(["thing","movie","person"]):
        out_results = json.dumps(_bot_results[i::3])
        out_embeddings = json.dumps(_bot_embeddings[i::3])

        with open(f"{_bot.engine_to_use}_{bot}_{_list}_result.txt","w") as f:
            f.write(out_results)
        with open(f"{_bot.engine_to_use}_{bot}_{_list}_embeddings.txt","w") as f:
            f.write(out_embeddings)
        print(f'{_list} done.')

In [10]:
create_save_bot_results_for_bot("bubblebot_v4",number=50,engine="curie")

thing done.
movie done.
person done.


In [ ]:
bot_bubblebot_v3_curie= HoaxAI.create_bot_with_personality("bubblebot_v3")
bot_bubblebot_v3_results,bot_bubblebot_v3_results_embeddings = \
    get_responses_for_bot_for_query_list(bot_bubblebot_v3_curie,default_query_list)
out = json.dumps(bot_bubblebot_v3_results)

In [6]:
with open("curie_bubblebot_v3_person_result.txt","w") as f:
    f.write(out)

In [4]:
bot_bubblebot_v4_curie= HoaxAI.create_bot_with_personality("bubblebot_v4")
bot_bubblebot_v4_results,bot_bubblebot_v4_results_embeddings = \
    get_responses_for_bot_for_query_list(bot_bubblebot_v4_curie,default_query_list)
out = json.dumps(bot_bubblebot_v4_results)

0 of 5
1 of 5
2 of 5
3 of 5
4 of 5


In [5]:
with open("curie_bubblebot_v4_person_result.txt","w") as f:
    f.write(out)

In [ ]:
out = json.dumps(bubble_results)
with open ("bubble_bot_results.txt","w") as f:
    f.write(out)



In [ ]:
# $$$ Warning Costs Money ---
# retrieve results from openapi
# cost est = 7000*50*.006 around $2 for curie
a= input("spend money?")
if "yes" in a:
    results,result_emb= generate_results(data[:50],bot_bubblebot_curie)

In [ ]:
results_thing = results[0::3]
results_person = results[1::3]
results_movie = results[2::3]

lists_to_print = {"thing": results_thing,
                  "person": results_person,
                  "movie": results_movie}

for t,l in lists_to_print.items():
    print(t)
    file_name =f'./curie_bubblebot_{t}_result.txt'
    print(file_name)
    with open(file_name,"a") as f:
        f.write(json.dumps(l))
    for i in l:
        print(i.replace("\n"," "))